<a href="https://colab.research.google.com/github/nicolasvazquez95/Aprendiendo_DeepLearning/blob/main/projects/03_BitPredict_TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BitPredict Project: Time series & Deep Learning
_Forecasting Bitcoin prices_ 💰

In [ ]:
## Get data (Bitcoin price data)
## Link: coindesk.com